### Bedtools Post Hoc Analysis

In [36]:
import pandas as pd
import math
import scipy
import numpy as np
import pyranges as pr
import os

In [40]:
ancestry = {'AFR': 0,'EUR': 1, 'NAT':3, 'UNK': 4}
appended_data = []

for bed_file in os.listdir('/home/oona/nsf_grfp/all_bed/intersect'):
    df = pr.read_bed('all_bed/HG00551_intersect.bed', as_df=True)
    df.Name = [ancestry[item] for item in df.Name]
    
    df['file_name'] = os.path.basename(bed_file)
    
    df['ThickStart'] = df['ThickStart'].map(lambda x: x.lstrip('seg'))
    df['ThickStart'] = pd.to_numeric(df['ThickStart'])  

    # compute the difference based on shift, get points where ancestry switches and where ibd segments end
    df['Ances_Switch'] = df['Name'].diff()
    df['IBD_Switch'] = df['ThickStart'].diff()

    # count the number of ancestry switches within a contained ibd segment
    appended_data.append(df.query('Ances_Switch != 0 and IBD_Switch == 0'))

appended_data = pd.concat(appended_data)
appended_data.to_csv('switch_counts.csv', index=False)  